# Aspect-based SA

Вспомним, какие есть подзадачи:
- выделение упоминаний аспектов
- классификация аспектов по категориям
- оценка тональности по отношению к аспекту

-- и всё это подозрительно напоминает наше проектное задание

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## preprocessing

Данные уже есть в tsv формате, можем пока их просто прочитать:

In [2]:
!pip install -U 'scikit-learn<0.24'

In [3]:
!pip install nltk pymorphy2

In [4]:
import pandas as pd

In [5]:
from nltk.tokenize import RegexpTokenizer
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')

def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

In [6]:
# !wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_aspects.txt
# !wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_reviews.txt

In [7]:
path = '/content/drive/MyDrive/NLProject/'
train_asp = pd.read_csv(
    path + 'train_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)
train_texts = pd.read_csv(path + 'train_reviews.txt', delimiter='\t', names=['text_id','text'])

In [9]:
train_asp.head(3)

,text_id,category,mention,start,end,sentiment
0,3976,Whole,ресторане,71,80,neutral
1,3976,Whole,ресторанах,198,208,neutral
2,3976,Whole,ресторане,256,265,neutral


In [10]:
train_texts.head(3)

,text_id,text
0,3976,"День 8-го марта прошёл, можно и итоги подвести..."
1,30808,Отмечали в этом ресторане день рождение на пер...
2,14031,Хочу поделиться своим впечатлением от посещени...


## unsupervised aspect extraction

Что можно сделать для извлечения аспектных слов, если у нас нет разметки?
Составим частотный словарь!

In [11]:
train_texts['lemmas'] = [normalize(text) for text in train_texts['text']]

In [12]:
all_lemmas = [l for tlemmas in train_texts['lemmas'] for l in tlemmas]

In [13]:
import nltk
fd = nltk.FreqDist(all_lemmas)

In [14]:
fd.most_common(100)

[('и', 1443),
 ('в', 1068),
 ('не', 947),
 ('быть', 682),
 ('очень', 588),
 ('мы', 585),
 ('что', 572),
 ('на', 541),
 ('с', 521),
 ('всё', 429),
 ('это', 414),
 ('ресторан', 361),
 ('но', 357),
 ('я', 350),
 ('то', 226),
 ('из', 220),
 ('как', 219),
 ('так', 216),
 ('хороший', 213),
 ('а', 210),
 ('понравиться', 206),
 ('к', 195),
 ('по', 193),
 ('блюдо', 189),
 ('кухня', 185),
 ('за', 184),
 ('место', 180),
 ('раз', 177),
 ('обслуживание', 172),
 ('интерьер', 169),
 ('ещё', 166),
 ('большой', 163),
 ('официант', 158),
 ('заведение', 156),
 ('для', 155),
 ('вкусный', 153),
 ('меню', 150),
 ('весь', 149),
 ('он', 144),
 ('есть', 139),
 ('столик', 126),
 ('принести', 123),
 ('они', 123),
 ('приятный', 121),
 ('сказать', 120),
 ('спасибо', 120),
 ('друг', 117),
 ('вкусно', 117),
 ('день', 114),
 ('наш', 114),
 ('салат', 112),
 ('у', 111),
 ('зал', 111),
 ('человек', 110),
 ('еда', 110),
 ('уютный', 109),
 ('заказать', 107),
 ('стол', 103),
 ('уже', 103),
 ('один', 103),
 ('от', 102),
 ('

### задание

Давайте оставим в частотном списке только существительные.

Возможно, стоит также добавить биграммы (например, ADJF+NOUN, NOUN+NOUN ...)

In [15]:
def get_nouns(text):
  nouns = []
  for word in tokenize(text):
    if word:
      parse = morph.parse(word)[0]
      if parse.tag.POS == 'NOUN':
        nouns.append(parse.normal_form)
  return nouns

In [16]:
train_texts['nouns'] = [get_nouns(text) for text in train_texts['text']]

In [17]:
all_nouns = [l for tlemmas in train_texts['nouns'] for l in tlemmas]
fd_nouns = nltk.FreqDist(all_nouns)

In [18]:
fd_nouns.most_common(10)

[('ресторан', 361),
 ('блюдо', 189),
 ('кухня', 185),
 ('место', 180),
 ('раз', 177),
 ('обслуживание', 172),
 ('интерьер', 169),
 ('официант', 158),
 ('заведение', 156),
 ('меню', 150)]

Как классифицировать полученные аспекты?

Попробуем подход на основе пословных эмбеддингов:

In [19]:
!pip install gensim

In [20]:
from gensim.models.keyedvectors import KeyedVectors

In [21]:
!wget https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz

--2021-12-27 11:56:37--  https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394697055 (376M) [application/x-gzip]
Saving to: ‘ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz’

ruwikiruscorpora_up 100%[===================>] 376.41M   181MB/s    in 2.1s    

2021-12-27 11:56:39 (181 MB/s) - ‘ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz’ saved [394697055/394697055]



In [22]:
!gzip -d ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz

gzip: ruwikiruscorpora_upos_skipgram_300_2_2018.vec already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [23]:
wv = KeyedVectors.load_word2vec_format('ruwikiruscorpora_upos_skipgram_300_2_2018.vec')

In [24]:
classes = ['Whole', 'Food', 'Interior', 'Service']
base_words = ['ресторан_NOUN', 'еда_NOUN', 'интерьер_NOUN', 'обслуживание_NOUN']
base_vectors = []
for c, word in zip(classes, base_words):
    base_vectors.append(wv[word])

Напишем функцию, выбирающую класс, базовый вектор которого ближе всего к заданному слову

In [25]:
import numpy as np
def get_most_similar(word):
    # YOUR CODE
    sim = wv.cosine_similarities(wv[word], base_vectors)
    return classes[np.argmax(sim)]

In [26]:
get_most_similar('блюдо_NOUN')

'Food'

In [27]:
for k, v in fd_nouns.most_common(100):
  key = k + '_NOUN'
  try:
    print(key, get_most_similar(key))
  except KeyError:
    pass

ресторан_NOUN Whole
блюдо_NOUN Food
кухня_NOUN Whole
место_NOUN Service
обслуживание_NOUN Service
интерьер_NOUN Interior
официант_NOUN Whole
заведение_NOUN Whole
меню_NOUN Whole
столик_NOUN Whole
друг_NOUN Food
день_NOUN Food
салат_NOUN Food
зал_NOUN Whole
человек_NOUN Food
еда_NOUN Food
стол_NOUN Food
минута_NOUN Food
заказ_NOUN Service
персонал_NOUN Service
время_NOUN Food
выбор_NOUN Service
цена_NOUN Food
вечер_NOUN Food
пиво_NOUN Food
официантка_NOUN Whole
порция_NOUN Food
впечатление_NOUN Interior
музыка_NOUN Interior
девушка_NOUN Food
гость_NOUN Whole
десерт_NOUN Food
атмосфера_NOUN Interior
отзыв_NOUN Service
компания_NOUN Whole
вкус_NOUN Food
мясо_NOUN Food
администратор_NOUN Whole
уровень_NOUN Service
банкет_NOUN Whole
напиток_NOUN Food
рождение_NOUN Whole
вино_NOUN Food
сервис_NOUN Service
подруга_NOUN Whole
повар_NOUN Food
счёт_NOUN Service
свадьба_NOUN Whole
муж_NOUN Food
год_NOUN Service
праздник_NOUN Food
суп_NOUN Food
обстановка_NOUN Interior
час_NOUN Food
паста_NOUN Foo

### ?
Что тут можно улучшить? (можно вспомнить контекстуальные эмбеддинги)

## supervised aspect extraction

Теперь давайте реализуем baseline, с которым будем сравнивать проекты:

- выделим все упоминания аспектов из разметки и посчитаем их частоты для каждой категории

In [28]:
train_asp.value_counts(subset=['mention', 'category'])

mention                  category
ресторан                 Whole       100
место                    Whole        97
обслуживание             Service      96
интерьер                 Interior     87
ресторане                Whole        75
                                    ... 
пирожными                Food          1
пирожок к первому блюду  Food          1
пицц                     Food          1
пиццами                  Food          1
 Цезарь                  Food          1
Length: 1955, dtype: int64

In [29]:
s = train_asp.value_counts(subset=['mention', 'category'])

In [30]:
s

mention                  category
ресторан                 Whole       100
место                    Whole        97
обслуживание             Service      96
интерьер                 Interior     87
ресторане                Whole        75
                                    ... 
пирожными                Food          1
пирожок к первому блюду  Food          1
пицц                     Food          1
пиццами                  Food          1
 Цезарь                  Food          1
Length: 1955, dtype: int64

In [31]:
train_counts = dict(zip(s.keys(), s.to_list()))

In [32]:
train_counts

{('ресторан', 'Whole'): 100,
 ('место', 'Whole'): 97,
 ('обслуживание', 'Service'): 96,
 ('интерьер', 'Interior'): 87,
 ('ресторане', 'Whole'): 75,
 ('заведение', 'Whole'): 65,
 ('Интерьер', 'Interior'): 63,
 ('блюда', 'Food'): 58,
 ('кухня', 'Food'): 52,
 ('цены', 'Price'): 42,
 ('официант', 'Service'): 39,
 ('официантка', 'Service'): 38,
 ('персонал', 'Service'): 37,
 ('Кухня', 'Food'): 37,
 ('Обслуживание', 'Service'): 35,
 ('официанты', 'Service'): 34,
 ('музыка', 'Interior'): 34,
 ('порции', 'Food'): 33,
 ('меню', 'Food'): 33,
 ('пиво', 'Food'): 32,
 ('еда', 'Food'): 31,
 ('атмосфера', 'Interior'): 31,
 ('девушка', 'Service'): 28,
 ('ресторана', 'Whole'): 28,
 ('заведении', 'Whole'): 24,
 ('администратор', 'Service'): 22,
 ('кафе', 'Whole'): 21,
 ('горячее', 'Food'): 21,
 ('десерт', 'Food'): 20,
 ('заведения', 'Whole'): 19,
 ('сервис', 'Service'): 19,
 ('Еда', 'Food'): 19,
 ('мясо', 'Food'): 18,
 ('блюд', 'Food'): 18,
 ('блюдо', 'Food'): 17,
 ('персоналу', 'Service'): 16,
 ('офици

In [33]:
train_asp['lemmas'] = [tuple(normalize(text)) for text in train_asp['mention']]

In [34]:
train_asp

,text_id,category,mention,start,end,sentiment,lemmas
0,3976,Whole,ресторане,71,80,neutral,"(ресторан,)"
1,3976,Whole,ресторанах,198,208,neutral,"(ресторан,)"
2,3976,Whole,ресторане,256,265,neutral,"(ресторан,)"
3,3976,Service,Столик бронировали,267,285,neutral,"(столик, бронировать)"
4,3976,Service,администратор,322,335,positive,"(администратор,)"
...,...,...,...,...,...,...,...
4758,16630,Service,обслуживание,85,97,positive,"(обслуживание,)"
4759,16630,Food,Еда,99,102,positive,"(еда,)"
4760,16630,Service,персоналу,244,253,positive,"(персонал,)"
4761,16630,Whole,ресторан,294,302,positive,"(ресторан,)"


In [35]:
train_asp.value_counts(subset=['lemmas', 'category'])

lemmas                     category
(ресторан,)                Whole       255
(обслуживание,)            Service     168
(интерьер,)                Interior    166
(официант,)                Service     143
(кухня,)                   Food        135
                                      ... 
(предварительный, заказ)   Service       1
(появляться,)              Service       1
(появиться,)               Service       1
(поступить,)               Service       1
(1450, 1750р, с, человек)  Price         1
Length: 1336, dtype: int64

### ?

Что тут можно улучшить?

In [36]:
train_asp

,text_id,category,mention,start,end,sentiment,lemmas
0,3976,Whole,ресторане,71,80,neutral,"(ресторан,)"
1,3976,Whole,ресторанах,198,208,neutral,"(ресторан,)"
2,3976,Whole,ресторане,256,265,neutral,"(ресторан,)"
3,3976,Service,Столик бронировали,267,285,neutral,"(столик, бронировать)"
4,3976,Service,администратор,322,335,positive,"(администратор,)"
...,...,...,...,...,...,...,...
4758,16630,Service,обслуживание,85,97,positive,"(обслуживание,)"
4759,16630,Food,Еда,99,102,positive,"(еда,)"
4760,16630,Service,персоналу,244,253,positive,"(персонал,)"
4761,16630,Whole,ресторан,294,302,positive,"(ресторан,)"


## sequence labelling

В лекции обсуждали, что задачу разметки аспектов можно представить как задачу разметки последовательности. Что для этого нужно? Что у нас уже есть?

1. Возьмем токенизатор, который сохраняет позиции токенов (stanza)
2. Сведём имеющуюся разметку к формату BIO:
  - для каждого текста возьмем список упоминаний аспектов и их позиции
  - пройдемся по токенам, сверим их с разметкой и припишем теги


In [37]:
# Отмечали в этом ресторане день рождение
# O        O O    B-Whole   O    O

In [38]:
!pip install stanza

In [39]:
import stanza

In [40]:
stanza.download('ru')

2021-12-27 11:59:08 INFO: Downloading default packages for language: ru (Russian)...
2021-12-27 11:59:10 INFO: File exists: /root/stanza_resources/ru/default.zip.
2021-12-27 11:59:20 INFO: Finished downloading models and saved to /root/stanza_resources.


In [41]:
# nlp = stanza.Pipeline('ru', processors='tokenize')
nlp = stanza.Pipeline('ru', processors='tokenize,pos')

2021-12-27 11:59:20 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2021-12-27 11:59:20 INFO: Use device: cpu
2021-12-27 11:59:20 INFO: Loading: tokenize
2021-12-27 11:59:20 INFO: Loading: pos
2021-12-27 11:59:21 INFO: Done loading processors!


In [42]:
# process text
nlp(train_texts['text'][0])

[
  [
    {
      "id": 1,
      "text": "День",
      "upos": "NOUN",
      "feats": "Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing",
      "start_char": 0,
      "end_char": 4
    },
    {
      "id": 2,
      "text": "8-го",
      "upos": "ADJ",
      "feats": "Case=Gen|Degree=Pos|Gender=Masc|Number=Sing",
      "start_char": 5,
      "end_char": 9
    },
    {
      "id": 3,
      "text": "марта",
      "upos": "NOUN",
      "feats": "Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing",
      "start_char": 10,
      "end_char": 15
    },
    {
      "id": 4,
      "text": "прошёл",
      "upos": "VERB",
      "feats": "Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act",
      "start_char": 16,
      "end_char": 22
    },
    {
      "id": 5,
      "text": ",",
      "upos": "PUNCT",
      "start_char": 22,
      "end_char": 23
    },
    {
      "id": 6,
      "text": "можно",
      "upos": "ADV",
      "feats": "Degree=Pos",
      "start_char": 24,
      "e

In [43]:
reviews = {}
with open(path + 'train_reviews.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    reviews[line[0]] = line[1]


reviews_test = {}
with open(path + 'dev_reviews.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    reviews_test[line[0]] = line[1]

In [44]:
from collections import defaultdict
aspects = defaultdict(list)

In [45]:
with open(path + 'train_aspects.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    keys = ('category', 'mention', 'start', 'end', 'sentiment')
    # ['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
    # тут можно отдельно запомнить начало и конец каждого упоминания
    aspects[line[0]].append(dict(zip(keys, line[1:])))

In [46]:
# здесь код для упоминаний из 1 токена
# ВАЖНО: для более длинных упоминаний нужно доделать ещё немного
markups = []
reviews_m = {}
poses = []
reviews_pos = {}
for text_id, text in reviews.items():
  processed = nlp(text)
  # print(processed)
  m = []
  r = []
  for token in processed.iter_tokens():
    # print(token.words[0].pos)
    add = False
    for mention in aspects[text_id]:
      if token.start_char == int(mention['start']) and token.end_char <= int(mention['end']):
      # if token.start_char == int(mention['start']) and token.end_char == int(mention['end']):
        # print(token.text, 'B-'+mention['category'])
        m.append('B-'+mention['category'])
        r.append(token.words[0].pos)
        add = True
        
      elif token.start_char > int(mention['start']) and token.end_char <= int(mention['end']):
        m.append('I-'+mention['category'])
        r.append(token.words[0].pos)
        add = True
      # if token.start_char == int(mention['start']) and token.end_char == int(mention['end']):
        # print(token.text, 'B-'+mention['category'])
        # m.append('B-'+mention['category'])
        # r.append(token.words[0].pos)
        # add = True
        # break
    if not add:
      # print(token.text, 'O')
      m.append( 'O')
      r.append(token.words[0].pos)
    reviews_m[text_id] = m
    markups.append(m)
    reviews_pos[text_id] = r
    poses.append(r)
  # break
print(markups[0])
print(poses[0])

print(markups[1])
print(poses[1])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Whole', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Whole', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Whole', 'O', 'B-Service', 'I-Service', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Service', 'O', 'O', 'B-Service', 'I-Service', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Whole', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Whole', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Food', 'I-Food', 'O', 'O', 'O', 'O', 'O', 'B-Food', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Service', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Service', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Interior', 'O', 'O', 'O', 'B-Interior', 'O', 'O', 'O', 'B-Interior', 'I-Interior', 'O', 'O', 'B-Interior', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [55]:
reviews_m.keys()

dict_keys(['3976', '30808', '14031', '2495', '38835', '1368', '7890', '6668', '9662', '18430', '14190', '2107', '3705', '33591', '34956', '3385', '33520', '12366', '29321', '22970', '35867', '36512', '36926', '1200', '15159', '18220', '36359', '13823', '1105', '32856', '32367', '6155', '37975', '32840', '23858', '10625', '25861', '4106', '27816', '18518', '36017', '29395', '27629', '10645', '34045', '6366', '8411', '18003', '19677', '11388', '28083', '6376', '11421', '24214', '31963', '3152', '27375', '10663', '25709', '719', '34607', '18780', '13225', '37516', '30744', '37473', '34282', '34232', '19265', '22975', '22015', '8555', '35486', '37819', '10942', '5037', '7824', '28745', '11825', '19503', '26330', '33693', '19383', '8759', '1434', '12203', '12880', '2073', '37611', '12341', '2692', '36049', '17572', '20784', '36483', '1751', '14568', '1032', '13100', '1511', '9036', '27841', '3452', '38077', '32040', '26887', '35613', '37220', '31004', '5648', '7079', '35635', '11267', '1052

In [56]:
markups_test = []
reviews_m_test = {}
poses_test = []
reviews_pos_test = {}
for text_id, text in reviews_test.items():
  processed = nlp(text)
  # print(processed)
  m = []
  r = []
  for token in processed.iter_tokens():
    # print(token.words[0].pos)
    add = False
    for mention in aspects[text_id]:
      if token.start_char == int(mention['start']) and token.end_char <= int(mention['end']):
      # if token.start_char == int(mention['start']) and token.end_char == int(mention['end']):
        # print(token.text, 'B-'+mention['category'])
        m.append('B-'+mention['category'])
        r.append(token.words[0].pos)
        add = True
        
      elif token.start_char > int(mention['start']) and token.end_char <= int(mention['end']):
        m.append('I-'+mention['category'])
        r.append(token.words[0].pos)
        add = True

    if not add:
      # print(token.text, 'O')
      m.append('O')
      r.append(token.words[0].pos)
    reviews_m_test[text_id] = m
    markups_test .append(m)
    reviews_pos_test [text_id] = r
    poses_test .append(r)

print(markups_test [0])
print(poses_test [0])

print(markups_test [1])
print(poses_test [1])

['O', 'O', 'B-Whole', 'I-Whole', 'I-Whole', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Service', 'B-Service', 'O', 'O', 'O', 'B-Service', 'O', 'B-Service', 'I-Service', 'I-Service', 'O', 'B-Service', 'I-Service', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Service', 'O', 'B-Service', 'I-Service', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Whole', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Service', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Food', 'I-Food', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Price', 'O', 'O', 'O', 'O', 'O', 'B-Whole', 'O', 'O', 'O', 'B-Food', 'O', 'O', 'B-Price', 'O', 'O', 'O', 'B-Service', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Whole', 'O', 'O', 'O', 'O', 'O', 'O']
['VERB', 'ADP', 'PUNCT', 'NOUN', 'PUNCT', 'ADV', 'PUNCT', 'PART', 'VERB', 'ADP', 'PRON', 'PU

## CRF

Популярный алгоритм для разметки последовательностей. Попробуем сначала на задаче NER с помощью [sklearn-crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/api.html)

Возьмём корпус WikiNER:

In [126]:
!pip install -U 'scikit-learn<0.24'

In [127]:
# from sklearn.model_selection import train_test_split

In [128]:
# !wget https://github.com/dice-group/FOX/raw/master/input/Wikiner/aij-wikiner-ru-wp3.bz2

In [129]:
# !pip install corus

In [130]:
# from corus import load_wikiner
# data = list(load_wikiner('aij-wikiner-ru-wp3.bz2'))

Посмотрим на разметку:

In [47]:
for key, value in reviews.items():
  print(key)
  print(value)
  print(reviews_m[key])
  break

3976
День 8-го марта прошёл, можно и итоги подвести. Решил написать отзыв о ресторане в котором отметили прекрасный весений праздник, прочитал отзывы edik077 и Rules77777и понял что либо мы были вразных ресторанах, либо у ребят что-то незаладилось. Но теперь о ресторане. Столик бронировали заранее и сделали так как предложил администратор т.е. сделали предварительный заказ, когда придя увидели полностью заполненый ресторан поняли что совет нам дали действительно правильный, в ресторане было человек 70-80, тут действительно горячее блюдо можно ждать весьма долго. Меню достаточно разнообразное и весьма вкусное, мне и моим друзьям понравилось всё что нам принесли, а принесли нам немало. Обслуживание может и не самое лучьшее в городе, но официанты делали всё что нужно. Должен сказать что ждать нам всёравно пришлось, не очень долго конечно, но всё-же. Интерьер хороший, удобные диваны.Очень хорошая музыкальная программа и весёлый ведущий. В общем я остался доволен своим выбором, и тем как мы

In [48]:
for key, value in reviews_test.items():
  print(key)
  print(value)
  print(reviews_m[key])
  break

13823
Зашли в"аппетит" случайно.Не смотря на то,что был будний день( вторник 14 сентября) и достаточно много народа, все же решили остаться.нас встретил менеджер- темноволосая стройная девушка, проводила к столу и дала меню...Спустя 5 минут пришла официантка, приняла заказ и удалилась...Посмотрев вокруг мы подумали что ждать придется долго- ресторан был полон, однако мы ошиблись , ожидание было недолгим- вполне приемлимым для любого заведения,за приятной беседой мы не заметили как прошло время...Бизнес ланч, на удивление, оказался очень вкусным и сытным- вполне соответствует своей цене. В общем очень хорошее место, разнообразное вкусное меню, приемлемые цены и высокое качество обслуживания! нам очень все понравилось. спасибо этому заведению! обязательно придем на выходных.
['O', 'O', 'B-Whole', 'I-Whole', 'I-Whole', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Service', 'B-Service', 'O', 'O', '

In [132]:
# data[0]

In [133]:
# data[0].tokens

In [134]:
# data = data[:100000]

Поделим датасет на train и test:

In [135]:
# train_sents, test_sents = train_test_split(data)

**Обучение CRF**

In [49]:
!pip install sklearn-crfsuite

In [50]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

Заведём экстрактор признаков и тегов:

In [58]:
reviews_m_test.keys()

dict_keys(['13823', '1427', '16714', '797', '34710', '33798', '2073', '785', '1434', '22386', '32442', '15423', '7193', '13100', '35635', '2107', '36085', '7824', '34045', '11421', '36126', '13186', '9721', '28083', '30744', '3976', '27040', '33591', '28258', '19503', '37391', '14568', '2495', '34402', '33524', '27375', '31071', '25960', '9837', '27841', '3231', '18780', '10798', '12131', '26152', '28112', '7011', '33043', '20862', '33252', '34956', '9289', '14539', '15567', '36926', '36512', '11532', '5155', '31004', '2606', '3552', '29395', '7154', '35325', '5974', '19817', '9216', '8996', '38299', '37819', '11770'])

In [57]:
sents = {}
for key, value in reviews.items():
  toks = []
  for i, tok in enumerate(value.split()):
    toks.append({'text': tok, 'tag':reviews_m[key][i], 'pos': reviews_pos[key][i]})
  sents[key] = {'tokens': toks}
  # print(key, value)
  # print(reviews_m[key])
  # print(reviews_pos[key])
  # break
# sents

sents_test = {}
for key, value in reviews_test.items():
  toks = []
  for i, tok in enumerate(value.split()):
    toks.append({'text': tok, 'tag':reviews_m_test[key][i], 'pos': reviews_pos_test[key][i]})
  sents_test[key] = {'tokens': toks}

In [59]:
def word2features(sent, i):
    # достаёт фичи для i-го токена в предложении
    word = sent[i]['text']
    postag = sent[i]['pos']
    
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1]['text']
        postag1 = sent[i-1]['pos']
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1]['text']
        postag1 = sent[i+1]['pos']
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    # достаёт фичи для всех токенов в предложении
    return [word2features(sent['tokens'], i) for i in range(len(sent['tokens']))]

def sent2labels(sent):
    return [label['tag'] for label in sent['tokens']]

def sent2tokens(sent):
    return [label['text'] for label in sent['tokens']]

In [60]:
for key, value in sents.items():
  print(key)
  print(value)
  print(sent2features(value))
  break

3976
{'tokens': [{'text': 'День', 'tag': 'O', 'pos': 'NOUN'}, {'text': '8-го', 'tag': 'O', 'pos': 'ADJ'}, {'text': 'марта', 'tag': 'O', 'pos': 'NOUN'}, {'text': 'прошёл,', 'tag': 'O', 'pos': 'VERB'}, {'text': 'можно', 'tag': 'O', 'pos': 'PUNCT'}, {'text': 'и', 'tag': 'O', 'pos': 'ADV'}, {'text': 'итоги', 'tag': 'O', 'pos': 'PART'}, {'text': 'подвести.', 'tag': 'O', 'pos': 'NOUN'}, {'text': 'Решил', 'tag': 'O', 'pos': 'VERB'}, {'text': 'написать', 'tag': 'O', 'pos': 'PUNCT'}, {'text': 'отзыв', 'tag': 'O', 'pos': 'VERB'}, {'text': 'о', 'tag': 'O', 'pos': 'VERB'}, {'text': 'ресторане', 'tag': 'O', 'pos': 'NOUN'}, {'text': 'в', 'tag': 'O', 'pos': 'ADP'}, {'text': 'котором', 'tag': 'B-Whole', 'pos': 'NOUN'}, {'text': 'отметили', 'tag': 'O', 'pos': 'ADP'}, {'text': 'прекрасный', 'tag': 'O', 'pos': 'PRON'}, {'text': 'весений', 'tag': 'O', 'pos': 'VERB'}, {'text': 'праздник,', 'tag': 'O', 'pos': 'ADJ'}, {'text': 'прочитал', 'tag': 'O', 'pos': 'NOUN'}, {'text': 'отзывы', 'tag': 'O', 'pos': 'NOU

In [61]:
X_train = [sent2features(value) for key, value in sents.items()]

In [62]:
for key, value in sents.items():
  print(key)
  print(value)
  print(sent2labels(value))
  break

3976
{'tokens': [{'text': 'День', 'tag': 'O', 'pos': 'NOUN'}, {'text': '8-го', 'tag': 'O', 'pos': 'ADJ'}, {'text': 'марта', 'tag': 'O', 'pos': 'NOUN'}, {'text': 'прошёл,', 'tag': 'O', 'pos': 'VERB'}, {'text': 'можно', 'tag': 'O', 'pos': 'PUNCT'}, {'text': 'и', 'tag': 'O', 'pos': 'ADV'}, {'text': 'итоги', 'tag': 'O', 'pos': 'PART'}, {'text': 'подвести.', 'tag': 'O', 'pos': 'NOUN'}, {'text': 'Решил', 'tag': 'O', 'pos': 'VERB'}, {'text': 'написать', 'tag': 'O', 'pos': 'PUNCT'}, {'text': 'отзыв', 'tag': 'O', 'pos': 'VERB'}, {'text': 'о', 'tag': 'O', 'pos': 'VERB'}, {'text': 'ресторане', 'tag': 'O', 'pos': 'NOUN'}, {'text': 'в', 'tag': 'O', 'pos': 'ADP'}, {'text': 'котором', 'tag': 'B-Whole', 'pos': 'NOUN'}, {'text': 'отметили', 'tag': 'O', 'pos': 'ADP'}, {'text': 'прекрасный', 'tag': 'O', 'pos': 'PRON'}, {'text': 'весений', 'tag': 'O', 'pos': 'VERB'}, {'text': 'праздник,', 'tag': 'O', 'pos': 'ADJ'}, {'text': 'прочитал', 'tag': 'O', 'pos': 'NOUN'}, {'text': 'отзывы', 'tag': 'O', 'pos': 'NOU

In [64]:
X_train = [sent2features(value) for _,value in sents.items()]
y_train = [sent2labels(value) for _,value in sents.items()]

X_test = [sent2features(value) for _,value in sents_test.items()]
y_test = [sent2labels(value) for _,value in sents_test.items()]

In [65]:
print(len(X_train[0]), len(X_train[1]), len(X_train[2]))

158 63 98


In [70]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [74]:
labels = list(crf.classes_)
labels.remove('O')
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9535563054608727

In [75]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-Food', 'I-Food', 'B-Interior', 'I-Interior', 'B-Price', 'I-Price', 'B-Service', 'I-Service', 'B-Whole', 'I-Whole'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

      B-Food      0.978     0.930     0.953       384
      I-Food      1.000     0.987     0.993       231
  B-Interior      1.000     0.925     0.961       146
  I-Interior      1.000     1.000     1.000        27
     B-Price      1.000     0.958     0.979        24
     I-Price      1.000     1.000     1.000         6
   B-Service      0.983     0.836     0.904       281
   I-Service      0.985     0.970     0.977        67
     B-Whole      0.966     0.927     0.947       124
     I-Whole      1.000     0.968     0.984        31

   micro avg      0.986     0.924     0.954      1321
   macro avg      0.991     0.950     0.970      1321
weighted avg      0.986     0.924     0.954      1321



In [76]:
labels = list(crf.classes_)
labels.remove('O')
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9535563054608727

In [78]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

      B-Food      0.978     0.930     0.953       384
      I-Food      1.000     0.987     0.993       231
  B-Interior      1.000     0.925     0.961       146
  I-Interior      1.000     1.000     1.000        27
     B-Price      1.000     0.958     0.979        24
     I-Price      1.000     1.000     1.000         6
   B-Service      0.983     0.836     0.904       281
   I-Service      0.985     0.970     0.977        67
     B-Whole      0.966     0.927     0.947       124
     I-Whole      1.000     0.968     0.984        31

   micro avg      0.986     0.924     0.954      1321
   macro avg      0.991     0.950     0.970      1321
weighted avg      0.986     0.924     0.954      1321



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-Food', 'I-Food', 'B-Interior', 'I-Interior', 'B-Price', 'I-Price', 'B-Service', 'I-Service', 'B-Whole', 'I-Whole'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


## + Подбор лучших гиперпараметров

In [85]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
# from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [86]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search - рандомизированный, чтобы колаб не отвалился
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 16.7min finished


CPU times: user 11min 44s, sys: 3.29 s, total: 11min 48s
Wall time: 16min 47s


In [88]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.20060494910360846, 'c2': 0.025813993669070386}
best CV score: 0.17303120178913833
model size: 1.10M


In [89]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-Food', 'I-Food', 'B-Interior', 'I-Interior', 'B-Price', 'I-Price', 'B-Service', 'I-Service', 'B-Whole', 'I-Whole'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

      B-Food      0.979     0.966     0.972       384
      I-Food      0.996     1.000     0.998       231
  B-Interior      1.000     0.945     0.972       146
  I-Interior      1.000     1.000     1.000        27
     B-Price      1.000     1.000     1.000        24
     I-Price      1.000     1.000     1.000         6
   B-Service      0.992     0.900     0.944       281
   I-Service      1.000     0.970     0.985        67
     B-Whole      0.975     0.960     0.967       124
     I-Whole      1.000     0.968     0.984        31

   micro avg      0.989     0.957     0.973      1321
   macro avg      0.994     0.971     0.982      1321
weighted avg      0.989     0.957     0.972      1321



In [90]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-Price -> I-Price 4.356653
O      -> O       3.978749
B-Price -> I-Price 3.819496
I-Whole -> I-Whole 3.707108
I-Food -> I-Food  3.604198
I-Interior -> I-Interior 3.578929
B-Interior -> I-Interior 3.318069
B-Service -> I-Service 3.316531
B-Food -> I-Food  3.253769
I-Service -> I-Service 2.989325
B-Whole -> I-Whole 2.565666
O      -> B-Food  1.716124
O      -> B-Whole 1.711296
O      -> B-Service 1.528208
O      -> B-Interior 1.222927
O      -> B-Price 1.129034
I-Service -> O       0.752802
B-Food -> O       0.638614
B-Service -> O       0.506112
I-Interior -> O       0.324695

Top unlikely transitions:
B-Food -> I-Service -2.764936
B-Whole -> I-Interior -2.848280
I-Service -> I-Food  -2.857876
B-Food -> B-Whole -2.941365
B-Service -> B-Interior -2.968983
B-Food -> I-Interior -3.019730
B-Whole -> B-Service -3.171863
B-Interior -> I-Food  -3.267092
B-Food -> I-Whole -3.275526
B-Service -> I-Food  -3.349889
B-Food -> B-Interior -3.556431
B-Whole -> I-Food  -3.61154

## См. также

[python-crfsuite](https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb) - без зависимости от версии scikit-learn

[BERT + CRF for NER](https://github.com/shushanxingzhe/transformers_ner)